In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# FraudFinder - BigQuery ML - Model Deployment

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/fraudfinder/raw/main/bqml/06_model_monitoring.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/fraudfinder/blob/main/bqml/06_model_monitoring.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/bqml/06_model_monitoring.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[FraudFinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the FraudFinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.

### Objective

In this notebook, you learn to deploy ML models and enable Vertex AI Model Monitoring service to detect feature skew and drift in the input predict requests. You will leverage the automatic generation of the input schema provided by Vertex AI Model Monitoring to analyze request data and detect feature skew and drift.


This tutorial uses the following Google Cloud services:

- [BigQuery](https://cloud.google.com/bigquery/)
- [Vertex AI](https://cloud.google.com/vertex-ai/)

The steps performed include:

- Configure Vertex Explainable AI for feature attribution skew and drift detection.
- Deploy the Vertex AI Model to a Vertex AI Endpoint.
- Define and create a Model Monitoring job.

### Costs 

This tutorial uses billable components of Google Cloud:

* BigQuery
* Vertex AI

Learn about [BigQuery Pricing](https://cloud.google.com/bigquery/pricing), [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [ ]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)

### Import libraries

In [ ]:
# General
from typing import Union, List, Dict
from datetime import datetime, timedelta
import time
import random
import pandas as pd

# BigQuery
from google.cloud import bigquery

# Vertex AI 
from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform import model_monitoring

### Define constants

In [ ]:
BQ_DATASET = "tx"
END_DATE_TRAIN = (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")
TRAIN_TABLE_NAME = f"train_table_{END_DATE_TRAIN.replace('-', '')}"
BQML_MODEL_ID = f"{PROJECT_ID}.{BQ_DATASET}.{MODEL_NAME}"
MODEL_ARTIFACT_URI = f"gs://{BUCKET_NAME}/deliverables/{MODEL_NAME}"
DEPLOY_VERSION = "tf2-cpu.2-5"
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)
DEPLOY_MACHINE_TYPE = "n1-standard-4"
MIN_REPLICA_COUNT = 1
MAX_REPLICA_COUNT = 1

### Initialize Vertex AI and BigQuery SDKs for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
vertex_ai.init(project=PROJECT_ID, location=REGION)

Create the BigQuery client.

In [ ]:
bq_client = bigquery.Client(project=PROJECT_ID)

### Monitor your model with Vertex AI Model Monitoring

With Vertex AI Model Monitoring, you can monitor for skew and drift detection of the predictions, features and its attributions (Explainable AI) in the incoming prediction requests.

With custom models, the model monitoring service requires:

- for drift detection, the schema of the features to derive the feature values

- for skew detection, a training data sample as baseline to calculate the distribution

- for feature attribution skew and drift detection, Vertex Explainable AI to be configured. 

In the following sections, we are going to cover all those requirements settings. 

#### Configure Vertex Explainable AI for feature attribution skew and drift detection

To configure Vertex Explainable AI for feature attribution skew and drift detection in our case, you need to define the explainability specification. Then, you need to pass the explainability specification to the model monitoring job. 

To define the explainability specification, you get the ML model schema from BQML model. Next you configure parameters for explaining model's predictions and metadata for describing expected model input and output. Please check out the [Vertex Explainable AI documentation](https://cloud.google.com/vertex-ai/docs/explainable-ai/configuring-explanations-feature-based#upload_model_xai_tf_sampled_shapley-gcloud) to know more about it. 

In [ ]:
bqml_model = bq_client.get_model(BQML_MODEL_ID)

In [ ]:
index_feature_mapping = []
for feature in bqml_model.feature_columns:
    index_feature_mapping.append(feature.name)
label_name = bqml_model.label_columns[0].name

In [ ]:
explanation_params =  vertex_ai.explain.ExplanationParameters({"sampled_shapley_attribution": {"path_count": 10}})
explanation_inputs = {feature_name:{'input_tensor_name':feature_name} for feature_name in index_feature_mapping}
explanation_outputs = {label_name: {'output_tensor_name': label_name}}
explanation_metadata = vertex_ai.explain.ExplanationMetadata(inputs=explanation_inputs, outputs=explanation_outputs)

#### Deploy the Vertex AI Model to a Vertex AI Endpoint

After you define the explainability specification, you can deploy the model previously trained using Vertex AI ML pipeline to a Vertex AI Endpoint. You get the model resource from the Vertex AI Model Registry. Then you deploy the model with the explainability specification in a new endpoint. 

In [ ]:
model = vertex_ai.Model.list(filter=f"display_name=bqml_fraud_classifier")[-1]

In [ ]:
endpoint = vertex_ai.Endpoint.create(display_name = f"{ENDPOINT_NAME}_monitored")

In [ ]:
model.deploy(
        endpoint=endpoint,
        deployed_model_display_name="fraud_detector_" + ID,
        machine_type=DEPLOY_MACHINE_TYPE,
        min_replica_count=MIN_REPLICA_COUNT,
        max_replica_count=MAX_REPLICA_COUNT,
        explanation_parameters=explanation_params,
        explanation_metadata=explanation_metadata,
        traffic_percentage = 100,
        sync=True
    )

#### Define and create a Model Monitoring job

To set up either skew detection or drift detection, create a model deployment monitoring job. 

The job requires the following specifications:

- `alert_config`: Configures how alerts are sent to the user. Right now only email alert is supported.
- `schedule_config`: Configures model monitoring job scheduling interval in hours. This defines how often the monitoring jobs are triggered.
- `logging_sampling_strategy`: Sample Strategy for logging.
- `drift_config` : Configures drift thresholds per each feature to monitor.
- `skew_config` : Configures skew thresholds per each feature to monitor.

##### Define the alerting configuration

The alerting configuration contains the mails to send alerts to. Also you can use the configuration to stream anomalies to Cloud Logging. 

In [ ]:
USER_EMAILS = ['recipient1@domain.com'] #'recipient1@domain.com', 'recipient2@domain.com'
alert_config = model_monitoring.EmailAlertConfig(USER_EMAILS, enable_logging=True)

##### Define the schedule configuration

The schedule configuration sets the hourly model monitoring job scheduling interval. 

In [ ]:
MONITOR_INTERVAL = 1
schedule_config = model_monitoring.ScheduleConfig(monitor_interval=MONITOR_INTERVAL)

##### Define the logging sample strategy

With the logging sample strategy, you configure how the model monitoring service randomly sample predictions to calculate monitoring metrics. The selected samples are logged to a BigQuery table. 

In [ ]:
SAMPLE_RATE = 0.5 
logging_sampling_strategy = model_monitoring.RandomSampleConfig(sample_rate=SAMPLE_RATE)

##### Define the drift detection configuration

With the drift detection configuration, you define the input features and the associated thresholds for monitoring feature distribution drift and feature attribution drift. 

In [ ]:
DRIFT_THRESHOLD_VALUE = 0.05
ATTRIBUTION_DRIFT_THRESHOLD_VALUE = 0.05

drift_thresholds = {
    "tx_amount": DRIFT_THRESHOLD_VALUE,
    "customer_id_nb_tx_1day_window": DRIFT_THRESHOLD_VALUE,
    "customer_id_avg_amount_1day_window": DRIFT_THRESHOLD_VALUE,
    "customer_id_nb_tx_15min_window": DRIFT_THRESHOLD_VALUE,
    "customer_id_avg_amount_15min_window": DRIFT_THRESHOLD_VALUE,
    "terminal_id_nb_tx_1day_window": DRIFT_THRESHOLD_VALUE,
    "terminal_id_risk_1day_window": DRIFT_THRESHOLD_VALUE,
    "terminal_id_nb_tx_15min_window": DRIFT_THRESHOLD_VALUE,
    "terminal_id_avg_amount_15min_window": DRIFT_THRESHOLD_VALUE
}

attribution_drift_thresholds = {
    "tx_amount": ATTRIBUTION_DRIFT_THRESHOLD_VALUE,
    "customer_id_nb_tx_1day_window": ATTRIBUTION_DRIFT_THRESHOLD_VALUE,
    "customer_id_avg_amount_1day_window": ATTRIBUTION_DRIFT_THRESHOLD_VALUE,
    "customer_id_nb_tx_15min_window": ATTRIBUTION_DRIFT_THRESHOLD_VALUE,
    "customer_id_avg_amount_15min_window": ATTRIBUTION_DRIFT_THRESHOLD_VALUE,
    "terminal_id_nb_tx_1day_window": ATTRIBUTION_DRIFT_THRESHOLD_VALUE,
    "terminal_id_risk_1day_window": ATTRIBUTION_DRIFT_THRESHOLD_VALUE,
    "terminal_id_nb_tx_15min_window": ATTRIBUTION_DRIFT_THRESHOLD_VALUE,
    "terminal_id_avg_amount_15min_window": ATTRIBUTION_DRIFT_THRESHOLD_VALUE
}

drift_config = model_monitoring.DriftDetectionConfig(
    drift_thresholds=drift_thresholds,
    attribute_drift_thresholds=attribution_drift_thresholds,
)

##### Define the skew detection configuration

With the skew detection configuration, you define the input features and the associated thresholds for monitoring feature distribution skew and feature attribution skew.

In [ ]:
TRAIN_DATA_SOURCE_URI = f"bq://{PROJECT_ID}.{BQ_DATASET}.{TRAIN_TABLE_NAME}"
TARGET = "tx_fraud"
SKEW_THRESHOLD_VALUE = 0.5
ATTRIBUTE_SKEW_THRESHOLD_VALUE = 0.5

skew_thresholds = {
    "tx_amount": SKEW_THRESHOLD_VALUE,
    "customer_id_nb_tx_1day_window": SKEW_THRESHOLD_VALUE,
    "customer_id_avg_amount_1day_window": SKEW_THRESHOLD_VALUE,
    "customer_id_nb_tx_15min_window": SKEW_THRESHOLD_VALUE,
    "customer_id_avg_amount_15min_window": SKEW_THRESHOLD_VALUE,
    "terminal_id_nb_tx_1day_window": SKEW_THRESHOLD_VALUE,
    "terminal_id_risk_1day_window": SKEW_THRESHOLD_VALUE,
    "terminal_id_nb_tx_15min_window": SKEW_THRESHOLD_VALUE,
    "terminal_id_avg_amount_15min_window": SKEW_THRESHOLD_VALUE
}

attribute_skew_thresholds = {
    "tx_amount": ATTRIBUTE_SKEW_THRESHOLD_VALUE,
    "customer_id_nb_tx_1day_window": ATTRIBUTE_SKEW_THRESHOLD_VALUE,
    "customer_id_avg_amount_1day_window": ATTRIBUTE_SKEW_THRESHOLD_VALUE,
    "customer_id_nb_tx_15min_window": ATTRIBUTE_SKEW_THRESHOLD_VALUE,
    "customer_id_avg_amount_15min_window": ATTRIBUTE_SKEW_THRESHOLD_VALUE,
    "terminal_id_nb_tx_1day_window": ATTRIBUTE_SKEW_THRESHOLD_VALUE,
    "terminal_id_risk_1day_window": ATTRIBUTE_SKEW_THRESHOLD_VALUE,
    "terminal_id_nb_tx_15min_window": ATTRIBUTE_SKEW_THRESHOLD_VALUE,
    "terminal_id_avg_amount_15min_window": ATTRIBUTE_SKEW_THRESHOLD_VALUE
}

skew_config = model_monitoring.SkewDetectionConfig(
    data_source=TRAIN_DATA_SOURCE_URI,
    skew_thresholds=skew_thresholds,
    attribute_skew_thresholds=attribute_skew_thresholds,
    target_field=TARGET,
)

##### Create the job configuration

Last step you create the Model monitoring job configuration

In [ ]:
explanation_config = model_monitoring.ExplanationConfig()

objective_config = model_monitoring.ObjectiveConfig(
    skew_detection_config=skew_config,
    drift_detection_config=drift_config,
    explanation_config=explanation_config,
)

##### Create the model monitoring job

Now you can create the model monitoring job.

In [ ]:
monitoring_job = vertex_ai.ModelDeploymentMonitoringJob.create(
    display_name="fraud_detection_" + ID,
    project=PROJECT_ID,
    location=REGION,
    endpoint=endpoint,
    logging_sampling_strategy=logging_sampling_strategy,
    schedule_config=schedule_config,
    alert_config=alert_config,
    objective_configs=objective_config,
)

##### Check the monitoring job state

You can check the status of the model monitoring job using the state attribute of the job instance.

In [ ]:
jobs = monitoring_job.list(filter=f"display_name=fraud_detection_{ID}")
job = jobs[0]
print(job.state)

#### Receiving email alert

After a minute or two, you should receive email at the address you configured above for `USER_EMAIL`. This email confirms successful deployment of your monitoring job.

#### Monitoring results in the Cloud Console

After one hour, you can examine your model monitoring data from the Cloud Console.

### END

Congrats! You successully finished the Fraudfinder lab series on how to build a real-time fraud detection system on Google Cloud. 

## (DO NOT RUN) Cleaning up

#### Delete the monitoring job

In [ ]:
# monitoring_job.pause()
# monitoring_job.delete()

#### Undeploy the model and delete the endpoint

In [ ]:
# endpoint.undeploy_all()
# endpoint.delete()